# sentinel-down-clip 🛰️🗺️

sentinel-down-clip is a Jupyter Notebook that simplifies the process of querying, downloading, and processing Sentinel satellite data. With its intuitive interface and comprehensive functionalities, it enables easy access to Sentinel data for various applications, including machine learning and deep learning.


### Installing dependencies : 

Python [3.8](https://www.python.org/ftp/python/3.8.0/python-3.8.0-amd64.exe) is used here, but you can use any version you want, just make sure it corresponds to the appropriate version of [gdal](https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal).
<br>
It is recommended to create a new virtual environement, to ensure a clean and isolated development environment. In your terminal run : 

In [ ]:
#python -m venv sdc_env
#sdc_env\Scripts\activate   

⚠️ Make sure to select your venv as kernel and install the packages, install ipykernel if necessary before running the cell.
<br>
🕓 It takes around 4min to install all the packages.
<br>


In [ ]:
# !pip install GDAL-3.2.3-cp38-cp38-win_amd64.whl
# !pip install keplergl sentinelsat numpy scikit-learn matplotlib 

### Downloading Sentinel data :

1. Run the code below to create a kepler map, navigate to the place of your choice and draw a polygon to fit the region of interest.
2. Don't hesitate to change the basemap if needed.
3. Then, select the polygon>right click>copy geometry.

In [ ]:
# Load an empty map
import keplergl as kp
import sentinelsat
map_1 = kp.KeplerGl()
map_1

Run this cell, paste the copied geometry, and give a name to it. It'll be saved in the rois folder, if you want to use them again.

In [ ]:
import os
import shutil

pasta = input("Paste the copied ROI here: ")
user_filename = input("Enter your desired filename: ")  

geojson_structure = '''
{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry":
'''

roi_geojson = geojson_structure + pasta + "\n}\n]}"

with open("roi.geojson", "w") as file:
    file.write(roi_geojson)

roi_folder = os.path.join("rois", user_filename+".geojson")

with open(roi_folder, "w") as new_roi_file:
    new_roi_file.write(roi_geojson)

print(f"New ROI saved as {roi_folder}.")



To query, check availibility and save footprints of Sentinel-2 MSIL1C data using [Sentinelsat](https://sentinelsat.readthedocs.io/en/stable/index.html), run this snippet. If no products are available just change the date, producttype or the cloud cover value.
<br>


In [ ]:
!sentinelsat -u username -p password -g roi.geojson -s 20210622 -e 20210628 --sentinel 2 --producttype S2MSI1C --cloud 0 -l 5 --footprints foot.geojson

We're now going to look at the footprint of the product(s) available, to check whether our ROI is within it, and tochoose which product to download.

In [ ]:
import keplergl as kp
with open('roi.geojson', 'r') as f1:
    roi = f1.read()

with open('foot.geojson', 'r') as f2:
    footprint = f2.read()

map_2 = kp.KeplerGl(height=600, width=800)
map_2.add_data(data=roi, name='roi')
map_2.add_data(data=footprint, name='footprint')
map_2


Create the folders in which to save your data.

In [ ]:
import os

folder_name = input('Enter the name of the folder: ')
downdir = os.path.join(os.getcwd(), 'downloads', folder_name)
cropdir = os.path.join(os.getcwd(), 'cropped', folder_name)
stackdir = os.path.join(os.getcwd(), 'stacked', folder_name)
clustdir = os.path.join(os.getcwd(), 'clustered', folder_name)

try:
    os.mkdir(downdir)
    print("New folder created at path:", downdir)
except FileExistsError:
    print("Download folder already exists at path:", downdir)

try:
    os.mkdir(cropdir)
    print("New folder created at path:", cropdir)
except FileExistsError:
    print("Crop folder already exists at path:", cropdir)

try:
    os.mkdir(stackdir)
    print("New folder created at path:", stackdir)
except FileExistsError:
    print("Stacking folder already exists at path:", stackdir)

try:
    os.mkdir(clustdir)
    print("New folder created at path:", clustdir)
except FileExistsError:
    print("Clustering folder already exists at path:", clustdir)


To download, just copy and paste the product name in the pname variable. Make sure to copy the right name. If the product is available it'll start downloading, if not, it'll be available 30min. Copernicus Open Access Hub’s quota currently permits users to request an offline product every 30 minutes.

In [ ]:
from sentinelsat import SentinelAPI

api = SentinelAPI('username', 'password', 'https://scihub.copernicus.eu/dhus')

# single file 

pname = 'db29e28a-9f0b-484f-ae92-a08a451dabf7'
product_info = api.get_product_odata(pname)
is_online = product_info['Online']
# or
is_online = api.is_online(pname)

if is_online:
    print(f'Product {pname} is online. Starting download..')
    api.download(pname,downdir)
    
else:
    print(f'Product {pname} is not online. Please retry in 30min.')
    api.trigger_offline_retrieval(pname)

# multiple files

# pname_list = []
# for pname in pname_list:
#     product_info = api.get_product_odata(pname)
#     is_online = product_info['Online']
#     # or
#     is_online = api.is_online(pname)

#     if is_online:
#         print(f'Product {pname} is online. Starting download..')
#         api.download(pname,downdir)
    
#     else:
#         print(f'Product {pname} is not online. Please retry in 30min.')
#         api.trigger_offline_retrieval(pname)

You can also download from the terminal using the following command :

In [ ]:
#sentinelsat -u username -p password  --path "{downdir}" -d --uuid 27733280-1504-4b01-9b99-8deffb9d9482

If you want to download all the results just add -d at the end of your first query.

### Croping Sentinel data to ROI using [Medusa-tb](https://github.com/aboulch/medusa_tb) :

Get the path of the downloaded data.

In [ ]:
import glob
product = glob.glob(os.path.join(downdir, '*.zip'))
if len(product) == 1:
    zip_path = product[0]
    print("Path of the product to crop :", zip_path)
else:
    print("Multiple zip files exist in the folder.")

Start clipping ✂️

In [ ]:
%run sclip.py --sentinel 2 --archive "{zip_path}" --dest "{cropdir}" --geojson roi.geojson 

### Clustering Sentinel data : optional

This part was adapted from the [Satellite_Imagery_Python](https://github.com/acgeospatial/Satellite_Imagery_Python) repository.

Importing the necessary libraries

In [ ]:
import numpy as np
from sklearn import cluster
from osgeo import gdal, gdal_array

Stacking the 13 bands into a single tif file.

In [ ]:
from osgeo import gdal

outvrt = 'vsimem/stacked.vrt' #/vsimem is special in-memory virtual "directory"


outtif = stackdir+'/'+folder_name +'_stacked.tif'

# To select single bands 
#tifs = ['..\T30SUC_20210627T105621_B01.tif', '..\T30SUC_20210627T105621_B02.tif',...]

#or for all tifs in a dir
import glob
tifs = glob.glob(cropdir+'/*.tif')
tifs = [tif for tif in tifs if not tif.endswith("TCI.tif")]

outds = gdal.BuildVRT(outvrt, tifs, separate=True)
outds = gdal.Translate(outtif, outds)

Now we turn the stacked product into numpy arrays.

In [ ]:
import numpy as np
import glob

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

product = glob.glob(os.path.join(stackdir, '*.tif'))


if len(product) == 1:
    tif_path = product[0]
    # Read in raster image 
    img_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)


    img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))

    for b in range(img.shape[2]):
        img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()
    
    new_shape = (img.shape[0] * img.shape[1], img.shape[2])
    print (img.shape)

    print (new_shape)


    X = img[:, :, :13].reshape(new_shape)

    print (X.shape)
else :
    print("Multiple tif files exist in the folder. Please check folder and retry.")  



Now fit it

In [ ]:
k_means = cluster.KMeans(n_clusters=8)
k_means.fit(X)

X_cluster = k_means.labels_


X_cluster = X_cluster.reshape(img[:, :, 0].shape)



And plot

In [ ]:
%matplotlib inline  

import matplotlib.pyplot as plt
print (X_cluster.shape)

plt.figure(figsize=(20,20))
plt.imshow(X_cluster, cmap="hsv")

plt.show()

Changing the classification is straight forward. In this example choose MiniBatchKMeans

In [ ]:
MB_KMeans = cluster.MiniBatchKMeans(n_clusters=8)
MB_KMeans.fit(X)

X_cluster = MB_KMeans.labels_


X_cluster = X_cluster.reshape(img[:, :, 0].shape)

Plot the result

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(X_cluster, cmap="hsv")

plt.show()

Finally save the result to a new geotiff

In [ ]:
from osgeo import gdal, gdal_array

## write out to tiff
 

ds = gdal.Open(outtif)
band = ds.GetRasterBand(1)
arr = band.ReadAsArray()
[cols, rows] = arr.shape

format = "GTiff"
driver = gdal.GetDriverByName(format)


outDataRaster = driver.Create(clustdir+'/'+folder_name+'_clust.gtif', rows, cols, 1, gdal.GDT_Byte)
outDataRaster.SetGeoTransform(ds.GetGeoTransform())##sets same geotransform as input
outDataRaster.SetProjection(ds.GetProjection())##sets same projection as input


outDataRaster.GetRasterBand(1).WriteArray(X_cluster)

outDataRaster.FlushCache() ## remove from memory
del outDataRaster ## delete the data (not the actual geotiff)